# Text data

### 1. Tokenizing text

---



In [1]:
# load the file --> the-verdict.txt
with open('the-verdict.txt', 'r', encoding = 'utf-8') as f:
    raw_text = f.read()

print(raw_text[:99])
print('Total characters:', len(raw_text))

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
Total characters: 20479


In [2]:
import re # regular expression module

In [3]:
# split on whitespaces
text = 'Hello, world. This, is a test.'
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [4]:
# split with punctuations and whitespaces
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [5]:
# split without whitespace --> better for LLMs
result = re.split(r'([,.]|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [6]:
# complex split
text = 'Hello, world. Is this-- a test?'
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [7]:
# split the entire story
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed)

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--', 'deploring', 'his', 'unaccountable', 'abdication', '.', '"', 'Of', 'course', 'it', "'", 's', 'going', 'to', 'send', 'the', 'value', 'of', 'my', 'picture', "'", 'way', 'up', ';', 'but', 'I', 'don', "'", 't', 'think', 'of', 'that', ',

In [8]:
# total tokens without whitespaces
len(preprocessed)

4690

In [9]:
# first 30 tokens for validation
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### 2. Tokens  --> TokenIDs

---



In [10]:
# creating vocab --> unique tokens and thier corresponding integers
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [11]:
# vocab
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)


In [12]:
# tokenizer class
class TokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [13]:
# instance of the class
tokenizer = TokenizerV1(vocab)
text = """It's the last he painted, you know," Mrs. Gisburn said with pardonable
pride."""
ids = tokenizer.encode(text)
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [14]:
print(tokenizer.decode(ids))

It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


### 3. Special Context Tokens

---



In [15]:
# extend list to special context
all_tokens = sorted(list((set(preprocessed))))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab))

1132


In [16]:
# validate
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [17]:
# tokenizer class
class TokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
         else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [18]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [19]:
tokenizer = TokenizerV2(vocab)
ids = tokenizer.encode(text)
print(ids)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


### 4. Byte-Pair Encoding

---



In [20]:
import tiktoken

In [21]:
# get GPT-2 bpe encoding
tokenizer = tiktoken.get_encoding('gpt2')

In [22]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
ids = tokenizer.encode(text, allowed_special = {'<|endoftext|>'})
print(ids)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [23]:
# sample text
sample_text = 'Akwirw ie'
ids = tokenizer.encode(sample_text)
print(ids)

[33901, 86, 343, 86, 37941]


In [24]:
tokenizer.decode(tokenizer.encode(sample_text))

'Akwirw ie'

### 5. Data Sampling using Sliding Window

---



In [25]:
with open("the-verdict.txt", "r") as f:
    raw_text = f.read()

print(len(raw_text))

20479


In [29]:
# tokenization using BPE
enc_text = tokenizer.encode(raw_text)
print("Tokens:", len(enc_text))

Tokens: 5145


In [31]:
# sample
enc_sample = enc_text[50:]
len(enc_sample)

5095

In [36]:
# input-target pairs
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [42]:
# validation
for i in range(1, context_size + 1):
    input = enc_sample[:i]
    target = enc_sample[i]
    print(tokenizer.decode(input), "-->", tokenizer.decode([target]))

 and -->  established
 and established -->  himself
 and established himself -->  in
 and established himself in -->  a


### 6. Implementing dataloaders in PyTorch

---

In [43]:
import torch
from torch.utils.data import Dataset, DataLoader

In [46]:
# Dataset class
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        # sliding window approach
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [51]:
# dataloader
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, stride = 12, shuffle = True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size, shuffle = True, drop_last = True)
    return dataloader

In [53]:
# load file
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size = 1, max_length = 4, stride = 1, shuffle = False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  616, 13674,  5891,   438]]), tensor([[13674,  5891,   438,  1092]])]


### 7. Token Embeddings


---



In [55]:
ids = torch.tensor([2, 3 , 5, 1])
print(ids)

tensor([2, 3, 5, 1])


In [56]:
# params
vocab_size = 6
output_dim = 3

In [58]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [63]:
# sample check
print(embedding_layer(torch.tensor([5])))

tensor([[-2.8400, -0.7849, -1.4096]], grad_fn=<EmbeddingBackward0>)


In [66]:
# token embedding of id
print(embedding_layer(torch.tensor(ids)))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


/tmp/ipython-input-2453163554.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(embedding_layer(torch.tensor(ids)))


### 8. Encoding Word Positions

---



In [68]:
# complex embeddings
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [71]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length = 4, stride = max_length, shuffle = False)
data_iter = iter(dataloader)
input, target = next(data_iter)
print("Token IDs:\n", input)
print("\nInputs shape:\n", input.shape)

Token IDs:
 tensor([[  262,  4269,    11, 22211],
        [  286,   465, 13476,     1],
        [ 5493,  2241,   318,   262],
        [    1,    40,  3114,   510],
        [ 3325,  1299,    11,  3888],
        [ 5469,   438, 14363,   938],
        [   13,   843,   340,   373],
        [  438, 14363, 10568,   852]])

Inputs shape:
 torch.Size([8, 4])


In [72]:
token_embeddings = token_embedding_layer(input)
print(token_embeddings.shape)

torch.Size([8, 4, 256])
